In [1]:
import os
import pickle
import numpy as np

from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16

from models import CNN_model
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

2024-04-18 15:01:41.737210: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 15:01:41.737273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 15:01:41.738232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 15:01:41.745402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 15:01:42.871839: W tensorflow/compiler/tf2

In [2]:
IMAGE_WIDTH = 448
IMAGE_HEIGHT = 448
MODELS_PATH: str = "./vision_models"
MODEL_NAME: str = "vgg16"
USE_GPU: bool = True

In [3]:
# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )
train_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train_prices: np.array = train1_df["price"].values

valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 311/311 [00:00<00:00, 1347550.15it/s]
Processing ../nybolig-scrape/output/train/train_2: 100%|██████████| 312/312 [00:00<00:00, 1843130.77it/s]
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 188.68it/s]


In [4]:
if USE_GPU:
    set_gpu()
else:
    set_cpu()

# Train Model
vgg_model, fit_history = CNN_model(
    VGG16,
    True,
    train_images,
    train_prices,
    valid_images,
    valid_prices,
)

2024-04-18 15:02:41.576684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-18 15:02:41.599218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-18 15:02:41.599283: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-18 15:02:41.602505: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-18 15:02:41.602601: I external/local_xla/xla/stream_executor

1 Physical GPUs, 1 Logical GPU
Epoch 1/150


2024-04-18 15:02:44.744050: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-18 15:02:44.862908: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-18 15:02:45.090753: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-18 15:02:45.735103: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-18 15:02:46.610674: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance 

 9/10 [==========================>...] - ETA: 0s - loss: 4082310.7500 - mean_absolute_error: 4081198.7500

2024-04-18 15:03:12.095407: W external/local_tsl/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


10/10 [==============================] - 49s 3s/step - loss: 4097393.0000 - mean_absolute_error: 4096285.0000 - val_loss: 4388010.0000 - val_mean_absolute_error: 4387135.5000
Epoch 2/150
10/10 [==============================] - 5s 468ms/step - loss: 4036192.0000 - mean_absolute_error: 4035239.0000 - val_loss: 4206546.5000 - val_mean_absolute_error: 4205321.0000
Epoch 3/150
10/10 [==============================] - 5s 478ms/step - loss: 3627990.5000 - mean_absolute_error: 3626500.5000 - val_loss: 3227507.7500 - val_mean_absolute_error: 3225537.5000
Epoch 4/150
10/10 [==============================] - 5s 478ms/step - loss: 2131933.2500 - mean_absolute_error: 2129590.2500 - val_loss: 1765978.8750 - val_mean_absolute_error: 1763165.3750
Epoch 5/150
 4/10 [===========>..................] - ETA: 2s - loss: 1999385.5000 - mean_absolute_error: 1996511.7500

In [ ]:
# Save model
if not os.path.exists(f"{MODELS_PATH}/{MODEL_NAME}"):
    os.makedirs(f"{MODELS_PATH}/{MODEL_NAME}")
vgg_model.save(f"{MODELS_PATH}/{MODEL_NAME}/model")

 # Save Training History
with open(f"{MODELS_PATH}/{MODEL_NAME}/history", "wb") as file_pi:
    pickle.dump(fit_history.history, file_pi)

# Export Model Architecture
plot_model(vgg_model, show_shapes=True, to_file=f"{MODELS_PATH}/{MODEL_NAME}/model_architecture.png")

INFO:tensorflow:Assets written to: ./vision_models/vgg16/model/assets


INFO:tensorflow:Assets written to: ./vision_models/vgg16/model/assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
